In [1]:
# 코드 8-8 사전 훈련된 인셉션 V3 모델 로드하기
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0) # 코델을 훈련하지 않습니다. 이 명령은 모든 훈련 연산을 비활성화합니다.

# 합성곱 기반 층만 사용한 인셉션 V3 네트워크를 만듭니다.
model = inception_v3.InceptionV3(weights="imagenet",
                                include_top=False)

87916544/87910968 [==============================] - 8s 0us/step


In [2]:
# 코드 8-9 딥드림 설정하기

# 층 이름과 계수를 매핑한 딕셔너리 입니다
# 최대화하려는 손실에 층의 활성화가 기여할 양을 정합니다
# 층 이름은 내장된 인셉션 V3 애플리케이션에 하드코딩되어 있는것입니다.
layer_contributions = {"mixed2":0.2,
                      "mixed3":3.,
                      "mixed4":2.,
                      "mixed5":1.5,
                      }

In [4]:
# 코드 8-10 최대화할 손실 정의하기
layer_dict = dict([(layer.name, layer) for layer in model.layers]) # 층 이름과 층 객체를 매핑한 딕셔너리를 만듭니다

loss = K.variable(0.) # 손실을 정의하고 각 층의 기여 분을 이 스칼라 변수에 추가할 것 입니다.
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output # 층의 출력을 얻습니다
    
    scaling = K.prod(K.cast(K.shape(activation), "float32"))
    
    # 층 특성의 L2 노름 제곱을 손실에 추가합니다. 이미지 테두리는 제외하고 손실에 추가합니다
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [7]:
# 코드 8-11 경사 상승법 과정
dream = model.input # 이 텐서는 생성된 딥드림 이미지를 저장합니다

grads = K.gradients(loss, dream)[0] # 손실에 대한 딥드림 이미지의 그래디언트를 계산합니다

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # 그래디언트를 정규화합니다(이 기교가 중요합니다)

# 주어진 입력 이미지에서 손실과 그래디언트 값을 계산할 케라스 Function 객체를 만듭니다
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# 이 함수는 경사 상승법을 여러 번 반복하여 수행합니다 
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print("...", i, "번째손실 :", loss_value)
        x += step * grad_values
    return x

RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.